In [1]:
from mysql_connection import get_cursor
import time
import pandas as pd
import numpy as np
from FoodItem import FoodItem


cursor = get_cursor()
# Start the timer
start = time.perf_counter()

In [2]:
user_id = (1,)
foodchecks = cursor.callproc("get_foodchecks_by_id", user_id)
checks = []
# stored_results is iterable, have to do this way
for result in cursor.stored_results():
    checks = result.fetchall()[0][1:]

print(checks)

(0, 0, 0, 0, 1, 1, 1, 1, 1)


In [3]:
incoming_data = {"latitude": "50.397120", "longitude": "-4.036932", "max-distance": "10", "isForChild": 0}
foodchecks = cursor.callproc("get_restaurants_within_distance", (incoming_data["latitude"], incoming_data["longitude"], incoming_data["max-distance"], "SUN", "12:00"))
df = None

for result in cursor.stored_results():
    df = pd.DataFrame(result)
    
df.columns = ["RestaurantID", "IsVegetarian", "IsVegan", "IsHalal", "IsKosher", "HasLactose", "HasNuts", "HasGluten", "HasEgg", "HasSoy", "FoodID", "FoodName", "IsChildMenu", "FoodTagID"]
print(df)

   RestaurantID  IsVegetarian  IsVegan  IsHalal  IsKosher  HasLactose  \
0             2             0        0        0         0           1   
1             2             0        0        0         0           1   
2             2             0        0        0         0           1   
3             2             0        0        0         0           1   
4             3             0        0        0         0           1   
5             3             0        0        0         0           1   
6             3             0        0        0         0           1   
7             2             0        0        0         0           1   
8             2             0        0        0         0           1   
9             2             0        0        0         0           1   

   HasNuts  HasGluten  HasEgg  HasSoy  FoodID                FoodName  \
0        1          1       1       1       1      Beyond Meat Burger   
1        1          1       1       1       1     

In [4]:
if checks[0]:
    df = df.drop(df[df["IsVegetarian"] != checks[0]].index)
if checks[1]:
    df = df.drop(df[df["IsVegan"] != checks[1]].index)
if checks[2]:
    df = df.drop(df[df["IsHalal"] != checks[2]].index)
if checks[3]:
    df = df.drop(df[df["IsKosher"] != checks[3]].index)
if checks[4]:
    df = df.drop(df[df["HasLactose"] != checks[4]].index)
if checks[5]:
    df = df.drop(df[df["HasNuts"] != checks[5]].index)
if checks[6]:
    df = df.drop(df[df["HasGluten"] != checks[6]].index)
if checks[7]:
    df = df.drop(df[df["HasEgg"] != checks[7]].index)
if checks[8]:
    df = df.drop(df[df["HasSoy"] != checks[8]].index)
if incoming_data["isForChild"]:
    df = df.drop(df[df["IsChildMenu"] != 1].index)

In [5]:
df

,RestaurantID,IsVegetarian,IsVegan,IsHalal,IsKosher,HasLactose,HasNuts,HasGluten,HasEgg,HasSoy,FoodID,FoodName,IsChildMenu,FoodTagID
0,2,0,0,0,0,1,1,1,1,1,1,Beyond Meat Burger,0,1
1,2,0,0,0,0,1,1,1,1,1,1,Beyond Meat Burger,0,4
2,2,0,0,0,0,1,1,1,1,1,1,Beyond Meat Burger,0,6
3,2,0,0,0,0,1,1,1,1,1,1,Beyond Meat Burger,0,7
4,3,0,0,0,0,1,1,1,1,1,2,Cornish Pastie,0,2
5,3,0,0,0,0,1,1,1,1,1,2,Cornish Pastie,0,3
6,3,0,0,0,0,1,1,1,1,1,2,Cornish Pastie,0,7
7,2,0,0,0,0,1,1,1,1,1,3,Grilled Haloumi Panini,0,5
8,2,0,0,0,0,1,1,1,1,1,3,Grilled Haloumi Panini,0,6
9,2,0,0,0,0,1,1,1,1,1,3,Grilled Haloumi Panini,0,7


In [6]:
food_items = []

for food_id in df.FoodID.unique():
    # For each unique food id
    # Extract that part of the dataframe, and create a new food item from it
    food_items.append(FoodItem(df[df["FoodID"] == food_id]))

In [7]:
end = time.perf_counter()
print(f"Done in {end - start:0.4f} seconds")

Done in 0.3486 seconds


In [8]:
for food_item in food_items:
    food_item.printInfo()

ID: 1 - (<class 'numpy.int64'>)
FoodTags: [1, 4, 6, 7] - (<class 'list'>)
Restaurant ID: 2
ID: 2 - (<class 'numpy.int64'>)
FoodTags: [2, 3, 7] - (<class 'list'>)
Restaurant ID: 3
ID: 3 - (<class 'numpy.int64'>)
FoodTags: [5, 6, 7] - (<class 'list'>)
Restaurant ID: 2
